In [2]:
!wget https://raw.githubusercontent.com/cbannard/lela60331_24-25/refs/heads/main/coursework/Compiled_Reviews.txt

--2025-01-17 13:43:41--  https://raw.githubusercontent.com/cbannard/lela60331_24-25/refs/heads/main/coursework/Compiled_Reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22322605 (21M) [text/plain]
Saving to: ‘Compiled_Reviews.txt’

Compiled_Reviews.tx 100%[===================>]  21.29M  80.0MB/s    in 0.3s    

2025-01-17 13:43:41 (80.0 MB/s) - ‘Compiled_Reviews.txt’ saved [22322605/22322605]



In [3]:
import numpy as np
import re
from collections import Counter
import matplotlib.pyplot as plt
import math

In [4]:
reviews=[]
sentiment_ratings=[]
product_types=[]
helpfulness_ratings=[]

with open("Compiled_Reviews.txt") as f:
   for line in f.readlines()[1:]:
        fields = line.rstrip().split('\t')
        reviews.append(fields[0])
        sentiment_ratings.append(fields[1])
        product_types.append(fields[2])
        helpfulness_ratings.append(fields[3])

In [5]:
token_def = re.compile(r"[\w]+|[^\w\s]")
tokenized_sents = [token_def.findall(txt) for txt in reviews]
tokens=[]
for s in tokenized_sents:
      tokens.extend(s)
counts=Counter(tokens)
so=sorted(counts.items(), key=lambda item: item[1], reverse=True)
so=list(zip(*so))[0]
type_list=so[0:10000]

In [7]:
# Create a 36547 x 10000 matrix of zeros
M = np.zeros((len(reviews), len(type_list)))
#iterate over the reviews
for i, rev in enumerate(reviews):
    # Tokenise the current review:
    tokens = set(token_def.findall(rev))
    # iterate over the words in our type list (the set of 5000 words):
    for j,t in enumerate(type_list):
        # if the current word j occurs in the current review i then set the matrix element at i,j to be one. Otherwise leave as zero.
        if t in tokens:
              M[i,j] = 1

In [8]:
train_ints=np.random.choice(len(reviews),int(len(reviews)*0.8),replace=False)
test_ints=list(set(range(0,len(reviews))) - set(train_ints))
M_train = M[train_ints,]
M_test = M[test_ints,]
labels = sentiment_ratings
labels_train = [labels[i] for i in train_ints]
labels_test = [labels[i] for i in test_ints]

In [ ]:
num_features=10000
y=[int(l == "positive") for l in labels_train]
weights = np.random.rand(num_features)
bias=np.random.rand(1)
n_iters = 7000
lr=0.07
logistic_loss=[]
num_samples=len(y)
for i in range(n_iters):
  z = M_train.dot(weights)+bias
  q = 1/(1+np.exp(-z))
  eps=0.00001
  loss = -sum((y*np.log2(q+eps)+(np.ones(len(y))-y)*np.log2(np.ones(len(y))-q+eps)))
  logistic_loss.append(loss)
  dw = ((q-y).dot(M_train) * (1/num_samples))
  db = sum((q-y))/num_samples
  weights = weights - lr*dw
  bias = bias - lr*db
plt.plot(range(1,n_iters),logistic_loss[1:])
plt.xlabel("number of epochs")
plt.ylabel("loss")
#loss = sum(-(np.ones(len(y))*np.log2(q)+(np.ones(len(y))-y)*np.log2(np.ones(len(y))-q)))

In [ ]:
# Make predictions on the test dataset using the trained logistic regression model
z = M_test.dot(weights)+bias
q = 1/(1+np.exp(-z))
y_test_pred=[int(ql > 0.5) for ql in q]

In [ ]:
# Calculate the accuracy of the performance of the model
y_test=[int(l == "positive") for l in labels_test]
acc_test=[int(yp == y_test[s]) for s,yp in enumerate(y_test_pred)]
print(sum(acc_test)/len(acc_test))

In [ ]:
labels_test_pred=["positive" if s == 1 else "negative" for s in y_test_pred]

true_positives=sum([int(yp == "positive" and labels_test[s] == "positive") for s,yp in enumerate(labels_test_pred)])
false_positives=sum([int(yp == "positive" and labels_test[s] == "negative") for s,yp in enumerate(labels_test_pred)])
false_negatives=sum([int(yp == "negative" and labels_test[s] == "positive") for s,yp in enumerate(labels_test_pred)])
precision_test=true_positives/(true_positives+false_positives)
recall_test=true_positives/(true_positives+false_negatives)
f1_score_test= 2 * (precision_test * recall_test) / (precision_test + recall_test)
print(f"test data precision:{precision_test}")
print(f"test data recall:{recall_test}")
print(f"test data F1-Score:{f1_score_test}")

In [ ]:
z = M_train.dot(weights)+bias
q = 1/(1+np.exp(-z))
y_train_pred=[int(ql > 0.5) for ql in q]
labels_train_pred=["positive" if s == 1 else "negative" for s in y_train_pred]

true_positives=sum([int(yp == "positive" and labels_train[s] == "positive") for s,yp in enumerate(labels_train_pred)])
false_positives=sum([int(yp == "positive" and labels_train[s] == "negative") for s,yp in enumerate(labels_train_pred)])
false_negatives=sum([int(yp == "negative" and labels_train[s] == "positive") for s,yp in enumerate(labels_train_pred)])
precision_train=true_positives/(true_positives+false_positives)
recall_train=true_positives/(true_positives+false_negatives)
f1_score_train= 2 * (precision_train * recall_train) / (precision_train + recall_train)
print(f"training data precision: {precision_train}")
print(f"training data recall: {recall_train}")
print(f"training data F1-Score:{f1_score_train}")

In [6]:
%%writefile README.md
# Sentiment Classification Project

## Project Description
This project implements logistic regression with one-hot encoding to classify the sentiment of Amazon product reviews.

## How to Run
1. Clone the repository: https://github.com/ydeng9950/LELA60331-Coursework
2. Run the first cell to download the raw dataset.
3. Run the second cell to load the necessary libraries.
4. Run the third cell to organize the data for preprocessing.
5. Run the fourth cell to tokenize the reviews, identify the 10000 most frequent words and create the vocabulary for one-hot encoding.
6. Run the fifth cell to create the one-hot encoded matrix.
7. Run the sixth cell to split the data into test and training set.
8. Run the seventh cell to train the logistic regression classifier on training data.
9. Run the eighth cell to make predictions on the test data.
10. Run the ninth cell to calculate the accuracy of the model.
11. Run the tenth cell to calculate the precision, recall and F1-score of the model on the test data.
12. Run the eleventh cell to calculate the precision, recall and F1-score of the model on the training data.

## References
The dataset used in this project was provided by Dr.Colin Bannard and can be accessed here: (https://colab.research.google.com/github/cbannard/lela60331_24-25/blob/main/coursework/coursework_data_import.ipynb).

Writing README.md
